In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
# import libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential 
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
# Helper libraries
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from google.colab import files
import io
from sklearn.utils import shuffle

# load mostly preprocessed data (preprocessing accomplished before 3/25 proposal)

# would use regular numpy syntax to load locally:
#X = np.load('<path>/X_cleaned_attempt_new.npy',allow_pickle=True)
#Y = np.load('<path>/Y_cleaned_attempt_new.npy',allow_pickle=True)
#X = np.asarray(X).astype('float32')
#Y = np.asarray(Y).astype('float32')

# i'm using colab, so it's slightly more complicated
ytrainval_byteseq = files.upload() # choose 'Y_cleaned_attempt_new.npy' from local system
ytrainval_filelike = io.BytesIO(ytrainval_byteseq['Y_cleaned_attempt_new.npy']) # create file-like object
ytrainval = np.load(ytrainval_filelike,allow_pickle=True) # create regular numpy array
xtrainval_byteseq = files.upload() # choose 'X_cleaned_attempt_new.npy' from local system
xtrainval_filelike = io.BytesIO(xtrainval_byteseq['X_cleaned_attempt_new.npy']) # create file-like object
xtrainval = np.load(xtrainval_filelike,allow_pickle=True) # create regular numpy array
ytest_byteseq = files.upload() # choose 'Y_test.npy' from local system
ytest_filelike = io.BytesIO(ytest_byteseq['Y_test.npy']) # create file-like object
ytest = np.load(ytest_filelike,allow_pickle=True) # create regular numpy array
xtest_byteseq = files.upload() # choose 'X_test.npy' from local system
xtest_filelike = io.BytesIO(xtest_byteseq['X_test.npy']) # create file-like object
xtest = np.load(xtest_filelike,allow_pickle=True) # create regular numpy array

#check shapes
print(xtrainval.shape)
print(ytrainval.shape)
print(xtest.shape)
print(ytest.shape)

#ytrainval needs to be converted to number representation of labels
feature_map = {'Rock':0,'Electronic':1,'Experimental':2,'Hip-Hop':3,'Folk':4,'Instrumental':5,'Pop':6,
              'International':7,'Classical':8,'Old-Time / Historic':9, 'Jazz':10,'Country':11,'Soul-RnB':12,
              'Spoken':13,'Blues':14,'Easy Listening':15}
for i in range(ytrainval.shape[0]):
    ytrainval[i] = float(feature_map[ytrainval[i]])
ytrainval[i] = np.asarray(ytrainval[i]).astype('float32')

print(ytrainval)
print(ytest)

# for the preliminary report, i did a roughly 80-20 split, but used the 20% as the training set to improve speed
# so will recombine and do a 90-10 split, using the 90% as training/validation
xfull = np.concatenate((xtrainval,xtest),axis=0)
yfull = np.concatenate((ytrainval,ytest),axis=0)
print(xfull.shape)
print(yfull.shape)



Saving Y_cleaned_attempt_new.npy to Y_cleaned_attempt_new.npy


Saving X_cleaned_attempt_new.npy to X_cleaned_attempt_new.npy


Saving Y_test.npy to Y_test.npy


Saving X_test.npy to X_test.npy
(9349, 520)
(9349,)
(40249, 520)
(40249,)
[3.0 3.0 3.0 ... 4.0 0.0 array(0., dtype=float32)]
[3.0 6.0 0.0 ... 0.0 0.0 array(0., dtype=float32)]
(49598, 520)
(49598,)


In [2]:
# this is where the strategy changes. We need to do a different train/test/val split
# let's see what the overall distribution between the categories is
yfull_1hot = to_categorical(yfull)
yfull_sum = np.sum(yfull_1hot, axis=0)
print(yfull_sum)


[14182.  9372. 10608.  3552.  2803.  2079.  2332.  1389.  1230.   554.
   571.   194.   175.   423.   110.    24.]


In [4]:
# In an extreme case, we might just take 20 of each class in the train set

yfull = yfull.reshape(yfull.shape[0],1)
both_full = np.concatenate((xfull, yfull), axis=1)
print(both_full.shape)
both_full = shuffle(both_full)
indices = [] 
num_counter = []
for i in range(16):
    num_counter.append(0)
for i in range(both_full.shape[0]):
    if num_counter[int(both_full[i,-1])] < 20:
        num_counter[int(both_full[i,-1])] += 1
        indices.append(i)
print(num_counter)
both_train_small = both_full[indices]
y_train_small = both_train_small[:,-1]
y_train_small_1hot = to_categorical(y_train_small).astype('float32')
x_train_small = both_train_small[:,:-1]
print(y_train_small_1hot.shape)
print(y_train_small.shape)
print(x_train_small.shape)

(49598, 521)
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
(320, 16)
(320,)
(320, 520)


In [ ]:
# need to implement k-fold cross validation
'''
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
# COMPILE NEW MODEL
model.compile(loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(
        learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
        metrics=['accuracy'])


# see https://medium.com/the-owl/k-fold-cross-validation-in-keras-3ec4a3a00538
from sklearn.model_selection import KFold, StratifiedKFold

kf = KFold(n_splits = x_trainval_small.shape[0])
for train_index, val_index in kf.split(np.zeros(x_trainval_small.shape[0]),y_trainval_small_1hot):
	
	
	
	
	# FIT THE MODEL
	history = model.fit(x_trainval_small,
			    epochs=10,,
			    validation_data=valid_data_generator)
	#PLOT HISTORY
	#		:
	#		:
	
	# LOAD BEST MODEL to evaluate the performance of the model
	model.load_weights("/saved_models/model_"+str(fold_var)+".h5")
	
	results = model.evaluate(valid_data_generator)
	results = dict(zip(model.metrics_names,results))
	
	VALIDATION_ACCURACY.append(results['accuracy'])
	VALIDATION_LOSS.append(results['loss'])
	
	tf.keras.backend.clear_session()
	
	fold_var += 1
  '''

In [31]:
'''
# let's give the datasets better names to reflect what i'm actually doing (kind of abandoned above plan)
import copy 
x_trainval_small = x_trainval_small.astype('float32')
y_trainval_small_1hot = y_trainval_small_1hot.astype('float32')
x_train_small = copy.deepcopy(x_trainval_small)
y_train_small_1hot = copy.deepcopy(y_trainval_small_1hot)
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_trainval_small, y_trainval_small_1hot, batch_size=32, epochs=100, validation_data=(x_trainval_small, y_trainval_small_1hot))
'''

Epoch 1/100
10/10 [==============================] - 1s 37ms/step - loss: 2.9122 - accuracy: 0.1406 - val_loss: 21.3840 - val_accuracy: 0.0719
Epoch 2/100
10/10 [==============================] - 0s 13ms/step - loss: 2.0922 - accuracy: 0.3375 - val_loss: 19.8783 - val_accuracy: 0.0625
Epoch 3/100
10/10 [==============================] - 0s 13ms/step - loss: 1.7584 - accuracy: 0.4719 - val_loss: 22.5311 - val_accuracy: 0.0625
Epoch 4/100
10/10 [==============================] - 0s 9ms/step - loss: 1.4792 - accuracy: 0.5562 - val_loss: 22.1548 - val_accuracy: 0.0625
Epoch 5/100
10/10 [==============================] - 0s 14ms/step - loss: 1.2724 - accuracy: 0.6094 - val_loss: 23.0078 - val_accuracy: 0.0625
Epoch 6/100
10/10 [==============================] - 0s 10ms/step - loss: 1.1068 - accuracy: 0.6875 - val_loss: 19.2610 - val_accuracy: 0.0625
Epoch 7/100
10/10 [==============================] - 0s 10ms/step - loss: 0.9062 - accuracy: 0.7437 - val_loss: 18.2128 - val_accuracy: 0.0625


In [36]:
'''
y_test = np.array([both_full[i,-1] for i in range(2000) if i not in indices])
x_test = np.array([both_full[i,:-1] for i in range(2000) if i not in indices])
y_test_1hot = to_categorical(y_test)
ytrain_sum = np.sum(y_train_small_1hot, axis=0)
print(y_test_1hot.shape)
print(x_test.shape)
ytest_sum = np.sum(y_test_1hot, axis=0)
print(ytest_sum)
print(yfull_sum)
print(ytrain_sum)
'''

(1747, 11)
(1747, 520)
[553. 382. 386. 119.  85.  67.  79.  24.  34.  12.   6.]
[14182.  9372. 10608.  3552.  2803.  2079.  2332.  1389.  1230.   554.
   571.   194.   175.   423.   110.    24.]
[20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]


In [10]:
val_numbers = [(i-20)//2 for i in yfull_sum] # number of each instance to put in val set 
val_indices = []
num_counter = []
for i in range(16):
    num_counter.append(0)
for i in range(both_full.shape[0]):
    if i not in indices: # don't count if already in training set
        if num_counter[int(both_full[i,-1])] < val_numbers[int(both_full[i,-1])]:
            num_counter[int(both_full[i,-1])] += 1
            val_indices.append(i)

#indices and val_indices should be different lists
for i in indices:
  for j in val_indices:
    assert i!=j

both_val_small = both_full[val_indices,:]
y_val_small = both_val_small[:,-1]
y_val_small_1hot = to_categorical(y_val_small).astype('float32')
x_val_small = both_val_small[:,:-1]
print(y_val_small_1hot.shape)
print(y_val_small.shape)
print(x_val_small.shape)

(24636, 16)
(24636,)
(24636, 520)


In [9]:
print(num_counter)

[7081, 4676, 5294, 1766, 1391, 1029, 1156, 684, 605, 267, 275, 87, 77, 201, 45, 2]


In [11]:
# now we need to put everything else in the test set
test_indices = []
for i in range(both_full.shape[0]):
    if i not in indices: # don't count if already in training set
        if i not in val_indices: # don't count if already in validation set
            test_indices.append(i)
both_test_small = both_full[test_indices,:]
y_test_small = both_test_small[:,-1]
y_test_small_1hot = to_categorical(y_test_small).astype('float32')
x_test_small = both_test_small[:,:-1]
print(y_test_small_1hot.shape)
print(y_test_small.shape)
print(x_test_small.shape)
assert y_test_small.shape[0] + y_val_small.shape[0] + y_train_small.shape[0] == both_full.shape[0]

(24642, 16)
(24642,)
(24642, 520)


In [16]:
from copy import deepcopy
#y_train_small_1hot_copy = deepcopy(y_train_small_1hot) float32 copy
#y_val_small_1hot_copy = deepcopy(y_val_small_1hot) float32 copy
#y_test_small_1hot_copy = deepcopy(y_test_small_1hot) float32 copy
x_train_small_copy = deepcopy(x_train_small) # float copy
x_val_small_copy = deepcopy(x_val_small) # float copy
x_test_small_copy = deepcopy(x_test_small) # float copy

x_train_small = x_train_small.astype('float32')
x_val_small = x_val_small.astype('float32')
x_test_small = x_test_small.astype('float32')

y_train_small_1hot = y_train_small_1hot.astype('int')
y_val_small_1hot = y_val_small_1hot.astype('int')
y_test_small_1hot = y_test_small_1hot.astype('int')

In [17]:
# now let's train!
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=10, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/10
10/10 [==============================] - 7s 327ms/step - loss: 2.9433 - accuracy: 0.1219 - val_loss: 8.8840 - val_accuracy: 0.2874
Epoch 2/10
10/10 [==============================] - 2s 255ms/step - loss: 2.0680 - accuracy: 0.3594 - val_loss: 5.7575 - val_accuracy: 0.2864
Epoch 3/10
10/10 [==============================] - 2s 252ms/step - loss: 1.7594 - accuracy: 0.4688 - val_loss: 5.6007 - val_accuracy: 0.2854
Epoch 4/10
10/10 [==============================] - 3s 290ms/step - loss: 1.4062 - accuracy: 0.5813 - val_loss: 4.7052 - val_accuracy: 0.2302
Epoch 5/10
10/10 [==============================] - 3s 289ms/step - loss: 1.2172 - accuracy: 0.6687 - val_loss: 5.0362 - val_accuracy: 0.0718
Epoch 6/10
10/10 [==============================] - 2s 255ms/step - loss: 1.0926 - accuracy: 0.7031 - val_loss: 4.4742 - val_accuracy: 0.0756
Epoch 7/10
10/10 [==============================] - 2s 251ms/step - loss: 0.9115 - accuracy: 0.7500 - val_loss: 4.7534 - val_accuracy: 0.0802
Epoch 

In [18]:
# clearly we need some strong regularization with this small training set
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=10, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/10
10/10 [==============================] - 5s 377ms/step - loss: 3.1246 - accuracy: 0.0719 - val_loss: 55.7526 - val_accuracy: 0.0418
Epoch 2/10
10/10 [==============================] - 2s 254ms/step - loss: 2.5676 - accuracy: 0.1844 - val_loss: 44.5348 - val_accuracy: 0.0418
Epoch 3/10
10/10 [==============================] - 2s 257ms/step - loss: 2.3043 - accuracy: 0.2656 - val_loss: 36.5491 - val_accuracy: 0.0418
Epoch 4/10
10/10 [==============================] - 2s 253ms/step - loss: 2.1448 - accuracy: 0.3688 - val_loss: 30.9776 - val_accuracy: 0.0418
Epoch 5/10
10/10 [==============================] - 3s 290ms/step - loss: 2.0008 - accuracy: 0.4062 - val_loss: 27.8499 - val_accuracy: 0.0417
Epoch 6/10
10/10 [==============================] - 3s 289ms/step - loss: 1.8829 - accuracy: 0.4563 - val_loss: 24.6718 - val_accuracy: 0.0416
Epoch 7/10
10/10 [==============================] - 2s 251ms/step - loss: 1.7615 - accuracy: 0.5031 - val_loss: 21.2001 - val_accuracy: 0.0417

In [20]:
# clearly we need some strong regularization with this small training set
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=200, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/200
10/10 [==============================] - 4s 290ms/step - loss: 3.2721 - accuracy: 0.0437 - val_loss: 77.1198 - val_accuracy: 0.2148
Epoch 2/200
10/10 [==============================] - 3s 290ms/step - loss: 2.7602 - accuracy: 0.1594 - val_loss: 66.4227 - val_accuracy: 0.2148
Epoch 3/200
10/10 [==============================] - 3s 290ms/step - loss: 2.5156 - accuracy: 0.1906 - val_loss: 55.4600 - val_accuracy: 0.2148
Epoch 4/200
10/10 [==============================] - 3s 290ms/step - loss: 2.3017 - accuracy: 0.2812 - val_loss: 47.9169 - val_accuracy: 0.2148
Epoch 5/200
10/10 [==============================] - 2s 251ms/step - loss: 2.1912 - accuracy: 0.3125 - val_loss: 41.2534 - val_accuracy: 0.2148
Epoch 6/200
10/10 [==============================] - 3s 290ms/step - loss: 2.0745 - accuracy: 0.3844 - val_loss: 36.3066 - val_accuracy: 0.2148
Epoch 7/200
10/10 [==============================] - 2s 251ms/step - loss: 1.9819 - accuracy: 0.4125 - val_loss: 32.1264 - val_accuracy:

In [21]:
# clearly we need some strong regularization with this small training set
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=200, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/200
10/10 [==============================] - 4s 320ms/step - loss: 3.0625 - accuracy: 0.0812 - val_loss: 122.4370 - val_accuracy: 0.1898
Epoch 2/200
10/10 [==============================] - 2s 241ms/step - loss: 2.8030 - accuracy: 0.1312 - val_loss: 90.4884 - val_accuracy: 0.1898
Epoch 3/200
10/10 [==============================] - 2s 233ms/step - loss: 2.6270 - accuracy: 0.1531 - val_loss: 69.8426 - val_accuracy: 0.1898
Epoch 4/200
10/10 [==============================] - 2s 239ms/step - loss: 2.5114 - accuracy: 0.1719 - val_loss: 53.6959 - val_accuracy: 0.1898
Epoch 5/200
10/10 [==============================] - 2s 244ms/step - loss: 2.3816 - accuracy: 0.2406 - val_loss: 44.6991 - val_accuracy: 0.1898
Epoch 6/200
10/10 [==============================] - 2s 236ms/step - loss: 2.2969 - accuracy: 0.2719 - val_loss: 38.1150 - val_accuracy: 0.1898
Epoch 7/200
10/10 [==============================] - 3s 289ms/step - loss: 2.2598 - accuracy: 0.2781 - val_loss: 31.9511 - val_accuracy

In [22]:
# the previous model is starting to look like a real neural network, but one that's overfitting
# maybe more complexity could also help it get toward 100% also, but overfitting is the main concern
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(8,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
10/10 [==============================] - 4s 302ms/step - loss: 91.2457 - accuracy: 0.1281 - val_loss: 110.9479 - val_accuracy: 0.0418
Epoch 2/100
10/10 [==============================] - 3s 290ms/step - loss: 71.2869 - accuracy: 0.2594 - val_loss: 86.3873 - val_accuracy: 0.0418
Epoch 3/100
10/10 [==============================] - 2s 261ms/step - loss: 54.1958 - accuracy: 0.3094 - val_loss: 61.3516 - val_accuracy: 0.0418
Epoch 4/100
10/10 [==============================] - 3s 290ms/step - loss: 40.3256 - accuracy: 0.3812 - val_loss: 49.5336 - val_accuracy: 0.0418
Epoch 5/100
10/10 [==============================] - 2s 266ms/step - loss: 29.5327 - accuracy: 0.4313 - val_loss: 38.7593 - val_accuracy: 0.0418
Epoch 6/100
10/10 [==============================] - 2s 268ms/step - loss: 21.5527 - accuracy: 0.4500 - val_loss: 28.5247 - val_accuracy: 0.0418
Epoch 7/100
10/10 [==============================] - 2s 262ms/step - loss: 15.6865 - accuracy: 0.4719 - val_loss: 23.3326 - val_a

In [23]:
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(8,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
10/10 [==============================] - 4s 307ms/step - loss: 91.3345 - accuracy: 0.1125 - val_loss: 142.0910 - val_accuracy: 0.0564
Epoch 2/100
10/10 [==============================] - 3s 290ms/step - loss: 72.1185 - accuracy: 0.2000 - val_loss: 97.7510 - val_accuracy: 0.0564
Epoch 3/100
10/10 [==============================] - 3s 290ms/step - loss: 55.7825 - accuracy: 0.2375 - val_loss: 68.9922 - val_accuracy: 0.0565
Epoch 4/100
10/10 [==============================] - 3s 290ms/step - loss: 42.2298 - accuracy: 0.2937 - val_loss: 49.8858 - val_accuracy: 0.0565
Epoch 5/100
10/10 [==============================] - 2s 267ms/step - loss: 31.4682 - accuracy: 0.3063 - val_loss: 35.9525 - val_accuracy: 0.0517
Epoch 6/100
10/10 [==============================] - 3s 291ms/step - loss: 23.3935 - accuracy: 0.3094 - val_loss: 25.3103 - val_accuracy: 0.0472
Epoch 7/100
10/10 [==============================] - 3s 290ms/step - loss: 17.3160 - accuracy: 0.3625 - val_loss: 17.8044 - val_a

In [25]:
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          #layers.Dense(8,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
10/10 [==============================] - 4s 291ms/step - loss: 91.2994 - accuracy: 0.1187 - val_loss: 145.3068 - val_accuracy: 0.0018
Epoch 2/100
10/10 [==============================] - 3s 289ms/step - loss: 72.6765 - accuracy: 0.3281 - val_loss: 98.0844 - val_accuracy: 0.0278
Epoch 3/100
10/10 [==============================] - 2s 257ms/step - loss: 57.1016 - accuracy: 0.4469 - val_loss: 76.6164 - val_accuracy: 0.0278
Epoch 4/100
10/10 [==============================] - 3s 290ms/step - loss: 44.3137 - accuracy: 0.5188 - val_loss: 55.7263 - val_accuracy: 0.0287
Epoch 5/100
10/10 [==============================] - 3s 290ms/step - loss: 33.8649 - accuracy: 0.6156 - val_loss: 43.5479 - val_accuracy: 0.0475
Epoch 6/100
10/10 [==============================] - 3s 290ms/step - loss: 25.7644 - accuracy: 0.5844 - val_loss: 36.5957 - val_accuracy: 0.0419
Epoch 7/100
10/10 [==============================] - 3s 290ms/step - loss: 19.5518 - accuracy: 0.6594 - val_loss: 33.0448 - val_a

In [26]:
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(4,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
10/10 [==============================] - 4s 328ms/step - loss: 90.6921 - accuracy: 0.0750 - val_loss: 133.5139 - val_accuracy: 8.1182e-05
Epoch 2/100
10/10 [==============================] - 2s 265ms/step - loss: 70.8027 - accuracy: 0.1562 - val_loss: 89.8773 - val_accuracy: 8.1182e-05
Epoch 3/100
10/10 [==============================] - 2s 268ms/step - loss: 53.9776 - accuracy: 0.1437 - val_loss: 64.3433 - val_accuracy: 8.1182e-05
Epoch 4/100
10/10 [==============================] - 3s 290ms/step - loss: 40.2068 - accuracy: 0.1813 - val_loss: 48.5809 - val_accuracy: 8.1182e-05
Epoch 5/100
10/10 [==============================] - 3s 290ms/step - loss: 29.5256 - accuracy: 0.2031 - val_loss: 35.2770 - val_accuracy: 8.1182e-05
Epoch 6/100
10/10 [==============================] - 2s 270ms/step - loss: 21.4615 - accuracy: 0.2031 - val_loss: 22.4706 - val_accuracy: 0.0614
Epoch 7/100
10/10 [==============================] - 3s 290ms/step - loss: 15.6559 - accuracy: 0.1813 - val_l

In [27]:
# val loss and train loss were pretty similar, can we add just a little more regularized complexity?
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(4,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.25),
          layers.Dense(16,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
10/10 [==============================] - 4s 331ms/step - loss: 90.6362 - accuracy: 0.0906 - val_loss: 80.1869 - val_accuracy: 0.0278
Epoch 2/100
10/10 [==============================] - 3s 279ms/step - loss: 71.1149 - accuracy: 0.1500 - val_loss: 62.4445 - val_accuracy: 0.0278
Epoch 3/100
10/10 [==============================] - 3s 291ms/step - loss: 54.7207 - accuracy: 0.1656 - val_loss: 47.5613 - val_accuracy: 0.0278
Epoch 4/100
10/10 [==============================] - 3s 292ms/step - loss: 41.1156 - accuracy: 0.1844 - val_loss: 35.5227 - val_accuracy: 0.0278
Epoch 5/100
10/10 [==============================] - 2s 272ms/step - loss: 30.3464 - accuracy: 0.1937 - val_loss: 26.2058 - val_accuracy: 0.0278
Epoch 6/100
10/10 [==============================] - 3s 291ms/step - loss: 22.0985 - accuracy: 0.1969 - val_loss: 19.1514 - val_accuracy: 0.0280
Epoch 7/100
10/10 [==============================] - 2s 271ms/step - loss: 15.9239 - accuracy: 0.2219 - val_loss: 13.7507 - val_ac

In [28]:
# val loss and train loss were pretty similar, can we add just a little more regularized complexity?
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(8,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.25),
          layers.Dense(16,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
10/10 [==============================] - 4s 333ms/step - loss: 91.1723 - accuracy: 0.0938 - val_loss: 88.4207 - val_accuracy: 7.3064e-04
Epoch 2/100
10/10 [==============================] - 2s 273ms/step - loss: 72.0327 - accuracy: 0.1688 - val_loss: 65.7303 - val_accuracy: 0.2081
Epoch 3/100
10/10 [==============================] - 2s 272ms/step - loss: 55.5954 - accuracy: 0.2313 - val_loss: 50.2828 - val_accuracy: 0.0078
Epoch 4/100
10/10 [==============================] - 3s 291ms/step - loss: 42.1046 - accuracy: 0.2219 - val_loss: 37.7902 - val_accuracy: 0.2148
Epoch 5/100
10/10 [==============================] - 3s 292ms/step - loss: 31.4439 - accuracy: 0.2344 - val_loss: 28.2855 - val_accuracy: 0.2148
Epoch 6/100
10/10 [==============================] - 3s 290ms/step - loss: 23.1820 - accuracy: 0.2500 - val_loss: 21.3509 - val_accuracy: 0.2148
Epoch 7/100
10/10 [==============================] - 3s 291ms/step - loss: 17.1382 - accuracy: 0.2656 - val_loss: 16.0098 - va

In [ ]:
'''
# val loss and train loss were pretty similar, can we add just a little more regularized complexity?
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(4,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))
'''

In [29]:
# further modifying the previous one that had the lowest validation loss
# clearly we need some strong regularization with this small training set
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/400
10/10 [==============================] - 3s 293ms/step - loss: 3.0775 - accuracy: 0.0781 - val_loss: 68.0999 - val_accuracy: 0.0285
Epoch 2/400
10/10 [==============================] - 2s 226ms/step - loss: 2.7480 - accuracy: 0.1344 - val_loss: 48.1763 - val_accuracy: 0.0326
Epoch 3/400
10/10 [==============================] - 3s 289ms/step - loss: 2.5762 - accuracy: 0.1750 - val_loss: 36.7923 - val_accuracy: 0.0283
Epoch 4/400
10/10 [==============================] - 3s 289ms/step - loss: 2.4501 - accuracy: 0.2313 - val_loss: 27.5479 - val_accuracy: 0.0276
Epoch 5/400
10/10 [==============================] - 2s 231ms/step - loss: 2.3886 - accuracy: 0.2469 - val_loss: 21.6546 - val_accuracy: 0.0224
Epoch 6/400
10/10 [==============================] - 2s 228ms/step - loss: 2.2769 - accuracy: 0.2781 - val_loss: 17.3950 - val_accuracy: 0.0085
Epoch 7/400
10/10 [==============================] - 3s 289ms/step - loss: 2.2218 - accuracy: 0.3313 - val_loss: 14.2242 - val_accuracy:

In [31]:
# further modifying the previous one that had the lowest validation loss
# clearly we need some strong regularization with this small training set
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/400
10/10 [==============================] - 4s 320ms/step - loss: 3.1920 - accuracy: 0.0906 - val_loss: 3.3433 - val_accuracy: 0.0606
Epoch 2/400
10/10 [==============================] - 3s 289ms/step - loss: 3.1685 - accuracy: 0.0781 - val_loss: 3.1523 - val_accuracy: 0.0585
Epoch 3/400
10/10 [==============================] - 2s 234ms/step - loss: 2.9601 - accuracy: 0.1156 - val_loss: 3.0727 - val_accuracy: 0.0617
Epoch 4/400
10/10 [==============================] - 3s 289ms/step - loss: 2.9290 - accuracy: 0.1094 - val_loss: 3.0011 - val_accuracy: 0.0696
Epoch 5/400
10/10 [==============================] - 3s 289ms/step - loss: 2.8302 - accuracy: 0.1219 - val_loss: 2.9387 - val_accuracy: 0.0809
Epoch 6/400
10/10 [==============================] - 3s 289ms/step - loss: 2.7886 - accuracy: 0.1437 - val_loss: 2.8875 - val_accuracy: 0.0989
Epoch 7/400
10/10 [==============================] - 2s 227ms/step - loss: 2.7031 - accuracy: 0.1437 - val_loss: 2.8468 - val_accuracy: 0.1124

KeyboardInterrupt: ignored

In [33]:
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(100,kernel_initializer='lecun_normal'),
          
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/400
10/10 [==============================] - 3s 268ms/step - loss: 3.2490 - accuracy: 0.0562 - val_loss: 3.5669 - val_accuracy: 0.0160
Epoch 2/400
10/10 [==============================] - 3s 289ms/step - loss: 2.9732 - accuracy: 0.1063 - val_loss: 3.0500 - val_accuracy: 0.0371
Epoch 3/400
10/10 [==============================] - 3s 289ms/step - loss: 2.8457 - accuracy: 0.1031 - val_loss: 2.8315 - val_accuracy: 0.0602
Epoch 4/400
10/10 [==============================] - 3s 289ms/step - loss: 2.7110 - accuracy: 0.1156 - val_loss: 2.7341 - val_accuracy: 0.0802
Epoch 5/400
10/10 [==============================] - 2s 232ms/step - loss: 2.6301 - accuracy: 0.1469 - val_loss: 2.6498 - val_accuracy: 0.1249
Epoch 6/400
10/10 [==============================] - 3s 289ms/step - loss: 2.6288 - accuracy: 0.1437 - val_loss: 2.5894 - val_accuracy: 0.1523
Epoch 7/400
10/10 [==============================] - 2s 236ms/step - loss: 2.5633 - accuracy: 0.1750 - val_loss: 2.5497 - val_accuracy: 0.1683

KeyboardInterrupt: ignored

In [34]:
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(100,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/400
10/10 [==============================] - 4s 273ms/step - loss: 3.1432 - accuracy: 0.0656 - val_loss: 2.6599 - val_accuracy: 0.2171
Epoch 2/400
10/10 [==============================] - 3s 289ms/step - loss: 3.0061 - accuracy: 0.0844 - val_loss: 2.6367 - val_accuracy: 0.1760
Epoch 3/400
10/10 [==============================] - 3s 290ms/step - loss: 2.8285 - accuracy: 0.0969 - val_loss: 2.6272 - val_accuracy: 0.1766
Epoch 4/400
10/10 [==============================] - 2s 251ms/step - loss: 2.7029 - accuracy: 0.1281 - val_loss: 2.6001 - val_accuracy: 0.1977
Epoch 5/400
10/10 [==============================] - 3s 289ms/step - loss: 2.6587 - accuracy: 0.1219 - val_loss: 2.5995 - val_accuracy: 0.2041
Epoch 6/400
10/10 [==============================] - 3s 289ms/step - loss: 2.5934 - accuracy: 0.1594 - val_loss: 2.5915 - val_accuracy: 0.2137
Epoch 7/400
10/10 [==============================] - 3s 290ms/step - loss: 2.6117 - accuracy: 0.1781 - val_loss: 2.5873 - val_accuracy: 0.2131

KeyboardInterrupt: ignored

In [35]:
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.6),
          layers.Dense(400,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/400
10/10 [==============================] - 4s 283ms/step - loss: 3.1426 - accuracy: 0.0594 - val_loss: 3.0280 - val_accuracy: 0.0669
Epoch 2/400
10/10 [==============================] - 3s 290ms/step - loss: 2.8878 - accuracy: 0.0656 - val_loss: 2.8016 - val_accuracy: 0.0908
Epoch 3/400
10/10 [==============================] - 2s 250ms/step - loss: 2.8074 - accuracy: 0.0750 - val_loss: 2.7517 - val_accuracy: 0.0898
Epoch 4/400
10/10 [==============================] - 3s 289ms/step - loss: 2.8508 - accuracy: 0.0812 - val_loss: 2.7287 - val_accuracy: 0.0645
Epoch 5/400
10/10 [==============================] - 3s 291ms/step - loss: 2.7985 - accuracy: 0.1031 - val_loss: 2.6880 - val_accuracy: 0.0813
Epoch 6/400
10/10 [==============================] - 3s 289ms/step - loss: 2.7599 - accuracy: 0.1094 - val_loss: 2.6184 - val_accuracy: 0.1076
Epoch 7/400
10/10 [==============================] - 3s 289ms/step - loss: 2.7240 - accuracy: 0.1250 - val_loss: 2.6207 - val_accuracy: 0.0761

KeyboardInterrupt: ignored

In [36]:
# so I created a model that isn't learning the training set! let's add another layer
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.6),
          layers.Dense(400,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(100,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/400
10/10 [==============================] - 5s 344ms/step - loss: 3.0883 - accuracy: 0.0688 - val_loss: 2.9835 - val_accuracy: 0.1045
Epoch 2/400
10/10 [==============================] - 3s 290ms/step - loss: 2.9620 - accuracy: 0.0906 - val_loss: 2.7528 - val_accuracy: 0.1849
Epoch 3/400
10/10 [==============================] - 3s 290ms/step - loss: 2.9926 - accuracy: 0.0594 - val_loss: 2.7380 - val_accuracy: 0.1481
Epoch 4/400
10/10 [==============================] - 3s 290ms/step - loss: 2.8372 - accuracy: 0.0812 - val_loss: 2.7412 - val_accuracy: 0.1210
Epoch 5/400
10/10 [==============================] - 2s 260ms/step - loss: 2.7822 - accuracy: 0.1094 - val_loss: 2.7443 - val_accuracy: 0.0612
Epoch 6/400
10/10 [==============================] - 3s 292ms/step - loss: 2.7500 - accuracy: 0.0938 - val_loss: 2.5819 - val_accuracy: 0.1909
Epoch 7/400
10/10 [==============================] - 2s 262ms/step - loss: 2.7912 - accuracy: 0.1031 - val_loss: 2.5309 - val_accuracy: 0.2255

KeyboardInterrupt: ignored

In [37]:
# that didn't help
# might need to lighten up on the dimensionality reduction
# so I created a model that isn't learning the training set! let's add another layer
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(40,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.6),
          layers.Dense(400,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/400
10/10 [==============================] - 4s 290ms/step - loss: 3.1774 - accuracy: 0.0625 - val_loss: 3.1198 - val_accuracy: 0.0790
Epoch 2/400
10/10 [==============================] - 2s 248ms/step - loss: 2.9016 - accuracy: 0.0906 - val_loss: 2.7817 - val_accuracy: 0.1790
Epoch 3/400
10/10 [==============================] - 2s 246ms/step - loss: 2.7968 - accuracy: 0.1219 - val_loss: 2.6586 - val_accuracy: 0.2015
Epoch 4/400
10/10 [==============================] - 3s 290ms/step - loss: 2.7957 - accuracy: 0.1187 - val_loss: 2.5915 - val_accuracy: 0.2062
Epoch 5/400
10/10 [==============================] - 3s 290ms/step - loss: 2.7343 - accuracy: 0.1375 - val_loss: 2.5691 - val_accuracy: 0.2094
Epoch 6/400
10/10 [==============================] - 3s 289ms/step - loss: 2.6231 - accuracy: 0.1437 - val_loss: 2.5490 - val_accuracy: 0.2063
Epoch 7/400
10/10 [==============================] - 3s 289ms/step - loss: 2.6170 - accuracy: 0.1625 - val_loss: 2.5445 - val_accuracy: 0.2076

KeyboardInterrupt: ignored

In [38]:
# that didn't help
# might need to lighten up on the dimensionality reduction
# so I created a model that isn't learning the training set! let's add another layer
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(40,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(40,kernel_initializer='lecun_normal',kernel_regularizer='l2'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.6),
          layers.Dense(400,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/400
10/10 [==============================] - ETA: 0s - loss: 3.5057 - accuracy: 0.0656     

KeyboardInterrupt: ignored

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


10/10 [==============================] - 109s 12s/step - loss: 3.5057 - accuracy: 0.0656 - val_loss: 3.6136 - val_accuracy: 0.0544
Epoch 2/400
10/10 [==============================] - 3s 292ms/step - loss: 3.3441 - accuracy: 0.0844 - val_loss: 3.3364 - val_accuracy: 0.0627
Epoch 3/400
10/10 [==============================] - 2s 267ms/step - loss: 3.1368 - accuracy: 0.1250 - val_loss: 3.0985 - val_accuracy: 0.1146
Epoch 4/400
10/10 [==============================] - 2s 264ms/step - loss: 3.1043 - accuracy: 0.1219 - val_loss: 3.0743 - val_accuracy: 0.1268
Epoch 5/400
10/10 [==============================] - 3s 292ms/step - loss: 3.0811 - accuracy: 0.1219 - val_loss: 3.0187 - val_accuracy: 0.1577
Epoch 6/400
10/10 [==============================] - 3s 290ms/step - loss: 2.9686 - accuracy: 0.1813 - val_loss: 2.9264 - val_accuracy: 0.2061
Epoch 7/400
10/10 [==============================] - 3s 290ms/step - loss: 2.9256 - accuracy: 0.1844 - val_loss: 2.8509 - val_accuracy: 0.2436
Epoch 8/400


KeyboardInterrupt

